In [1]:
from mofapy2.run.entry_point import entry_point
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing

In [2]:
samples = pd.read_csv("../data/samples/SAMPLES_Excel.tsv", sep="\t")
samples['label'] = samples.apply(lambda x: 0 if x['TMM'] == 'TEL' else 1
                                 if (x['WestATRX'] == "Neg" or x['WestDAXX'] == "Neg") else "AP",
                                 axis=1)
mapping = dict(zip(samples['Immortal'].values, samples['label'].values))

In [22]:
tmp = samples[samples['Immortal'].str.contains("JF")
              & (samples['Immortal'] != "JFCF_6_P_pLKO_5")][[
                  'Immortal', 'TMM'
              ]].rename(columns={'Immortal': 'sample'})
tmp['isALT'] = tmp['TMM'].map(lambda x: 1 if x == "ALT" else 0)
tmp[['sample','isALT']].to_csv("../data/multiomic/sample_TMM.csv", index=False)

In [108]:
cnv = pd.read_csv("../data/multiomic/CNV_JFCF-6_20200730.csv")

In [3]:
protein_sample = pd.read_csv(
    "../data/multiomic/Protein_JFCF-6_20200730.csv").set_index(['Gene']).T
rna_sample = pd.read_csv(
    "../data/multiomic/RNA_JFCF-6_20200730.csv").set_index(['gene']).T
cnv_sample = pd.read_csv("../data/multiomic/CNV_JFCF-6_20200730.csv").drop(
    ['Gene ID', 'Cytoband'], axis=1).set_index(['Gene Symbol']).T

In [6]:
rna_sample.index.name = 'sample'
rna_sample = rna_sample.reset_index()

In [8]:
rna_sample['sample'] = rna_sample['sample'].map(
    lambda x: x.replace('-', '_').replace('.', '_').replace('/', '_'))
rna_sample['sample'] = rna_sample['sample'].map(
    lambda x: x.replace('JFCF_6_T_1_P(ALT)', 'JFCF_6_T_1_P_ALT').replace(
        'JFCF_6_T_1_P(TEL)', '_JFCF_6_T_1_P_TEL'))
rna_sample.loc[rna_sample['sample'] == 'JFCF_6_T_1_P',
                  'sample'] = 'JFCF_6_T_1_P_TEL'

In [10]:
rna_sample.to_csv("../data/multiomic/RNA_JFCF-6_20200730_processed.csv",index=False)

In [24]:
protein_sample.index.name = 'sample'
protein_sample = protein_sample.reset_index()

In [25]:
protein_sample['sample'] = protein_sample['sample'].map(
    lambda x: x.replace('-', '_').replace('.', '_').replace('/', '_'))
protein_sample['sample'] = protein_sample['sample'].map(
    lambda x: x.replace('JFCF_6_T_1_P(ALT)', 'JFCF_6_T_1_P_ALT').replace(
        'JFCF_6_T_1_P(TEL)', '_JFCF_6_T_1_P_TEL'))
protein_sample.loc[protein_sample['sample'] == 'JFCF_6_T_1_P',
                  'sample'] = 'JFCF_6_T_1_P_TEL'

In [27]:
protein_sample.to_csv("../data/multiomic/Protein_JFCF-6_20200730_processed.csv",index=False)

In [110]:
protein_sample.index.name='sample'
rna_sample.index.name='sample'

cnv_sample.index.name='sample'

In [111]:
protein_mofa = pd.melt(protein_sample.reset_index(), id_vars='sample', var_name='feature')
protein_mofa['view'] = 'protein'
protein_mofa['feature'] = protein_mofa['feature'].map(lambda x:f"{x}_protein")
protein_mofa['value'] = protein_mofa['value'].map(np.log1p)

rna_mofa = pd.melt(rna_sample.reset_index(), id_vars='sample', var_name='feature')
rna_mofa['view'] = 'RNA'
rna_mofa['feature'] = rna_mofa['feature'].map(lambda x:f"{x}_rna")
rna_mofa['value'] = rna_mofa['value'].map(np.log1p)

cnv_mofa = pd.melt(cnv_sample.reset_index(), id_vars='sample', var_name='feature')
cnv_mofa['view'] = 'CNV'
cnv_mofa['feature'] = cnv_mofa['feature'].map(lambda x:x.replace("|", "_"))
cnv_mofa['feature'] = cnv_mofa['feature'].map(lambda x:f"{x}_cnv")


In [112]:
tmm_mofa = pd.DataFrame({'sample':protein_mofa['sample'].unique()})
tmm_mofa['view'] = 'TMM'
tmm_mofa['feature'] = 'isALT'
tmm_mofa['value'] = 0

In [113]:
combined_mofa = pd.concat([cnv_mofa, rna_mofa, protein_mofa, tmm_mofa])
combined_mofa['sample'] = combined_mofa['sample'].map(
    lambda x: x.replace('-', '_').replace('.', '_').replace('/', '_'))
combined_mofa['sample'] = combined_mofa['sample'].map(
    lambda x: x.replace('JFCF_6_T_1_P(ALT)', 'JFCF_6_T_1_P_ALT').replace(
        'JFCF_6_T_1_P(TEL)', '_JFCF_6_T_1_P_TEL'))
combined_mofa.loc[combined_mofa['sample'] == 'JFCF_6_T_1_P',
                  'sample'] = 'JFCF_6_T_1_P_TEL'

In [114]:
combined_mofa = combined_mofa[combined_mofa['sample'] != 'JFCF_6'].reset_index(
    drop=True)

In [115]:
combined_mofa.loc[combined_mofa['view'] == 'TMM',
                  'value'] = combined_mofa.loc[combined_mofa['view'] == 'TMM',
                                               'sample'].map(mapping)

In [116]:
combined_mofa['value'] = combined_mofa['value'].astype(float)

In [117]:
combined_mofa['group'] = 'group0'

In [118]:
combined_mofa = combined_mofa.drop_duplicates(["group","view","feature","sample"])

# train MOFA

In [119]:
ent = entry_point()
ent.set_data_options(
    scale_groups = False, 
    scale_views = True
)


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Scaling views to unit variance...



In [120]:
ent.set_data_df(combined_mofa, likelihoods = ["gaussian","gaussian", "bernoulli", 'gaussian'])



Loaded group='group0' view='CNV' with N=17 samples and D=24324 features...
Loaded group='group0' view='RNA' with N=17 samples and D=18836 features...
Loaded group='group0' view='TMM' with N=17 samples and D=1 features...
Loaded group='group0' view='protein' with N=17 samples and D=4553 features...





In [121]:
ent.set_model_options(
    factors = 10, 
    spikeslab_weights = True, 
    ard_factors = True,
    ard_weights = True
)

Model options:
- Automatic Relevance Determination prior on the factors: True
- Automatic Relevance Determination prior on the weights: True
- Spike-and-slab prior on the factors: False
- Spike-and-slab prior on the weights: True 

Likelihoods:
- View 0 (CNV): gaussian
- View 1 (RNA): gaussian
- View 2 (TMM): bernoulli
- View 3 (protein): gaussian


In [122]:
ent.set_train_options(
    iter = 1000, 
    convergence_mode = "fast", 
    startELBO = 1, 
    freqELBO = 1, 
    dropR2 = 0.001, 
    gpu_mode = False, 
    verbose = False, 
    seed = 1
)

In [123]:
ent.build()

ent.run()





######################################
## Training the model with seed 1 ##
######################################


ELBO before training: -7518583.29 

Iteration 1: time=0.18, ELBO=-1593420.00, deltaELBO=5925163.291 (78.80691162%), Factors=9
Iteration 2: time=0.14, ELBO=-1464550.73, deltaELBO=128869.273 (1.71401004%), Factors=9
Iteration 3: time=0.13, ELBO=-1400614.50, deltaELBO=63936.233 (0.85037607%), Factors=9
Iteration 4: time=0.13, ELBO=-1359388.88, deltaELBO=41225.612 (0.54831622%), Factors=9
Iteration 5: time=0.13, ELBO=-1333892.16, deltaELBO=25496.727 (0.33911610%), Factors=9
Iteration 6: time=0.13, ELBO=-1317105.73, deltaELBO=16786.432 (0.22326589%), Factors=9
Iteration 7: time=0.13, ELBO=-1304534.81, deltaELBO=12570.914 (0.16719791%), Factors=9
Iteration 8: time=0.14, ELBO=-1294844.06, deltaELBO=9690.751 (0.12889065%), Factors=9
Iteration 9: time=0.12, ELBO=-1286936.44, deltaELBO=7907.624 (0.10517439%), Factors=9
Iteration 10: time=0.12, ELBO=-1279906.23, deltaELBO=7030.

Iteration 94: time=0.14, ELBO=-1228366.37, deltaELBO=20.809 (0.00027677%), Factors=9
Iteration 95: time=0.14, ELBO=-1228345.92, deltaELBO=20.448 (0.00027197%), Factors=9
Iteration 96: time=0.12, ELBO=-1228325.82, deltaELBO=20.102 (0.00026737%), Factors=9
Iteration 97: time=0.12, ELBO=-1228306.05, deltaELBO=19.766 (0.00026290%), Factors=9
Iteration 98: time=0.12, ELBO=-1228286.61, deltaELBO=19.436 (0.00025850%), Factors=9
Iteration 99: time=0.13, ELBO=-1228267.50, deltaELBO=19.110 (0.00025417%), Factors=9
Iteration 100: time=0.13, ELBO=-1095674.49, deltaELBO=132593.016 (1.76353723%), Factors=9
Iteration 101: time=0.13, ELBO=-1094167.55, deltaELBO=1506.941 (0.02004289%), Factors=9
Iteration 102: time=0.14, ELBO=-1093391.36, deltaELBO=776.189 (0.01032361%), Factors=9
Iteration 103: time=0.13, ELBO=-1092979.51, deltaELBO=411.845 (0.00547769%), Factors=9
Iteration 104: time=0.13, ELBO=-1092759.51, deltaELBO=220.000 (0.00292608%), Factors=9
Iteration 105: time=0.12, ELBO=-1092643.50, deltaEL

In [124]:
ent.save("../results/MOFA_3omic.hdf5")


Saving model in ../results/MOFA_3omic.hdf5...



# MOFA analysis

In [85]:
cnv = pd.read_csv("../data/multiomic/CNV_JFCF-6_20200730.csv")
chr_map = cnv.set_index('Gene Symbol').to_dict()['Cytoband']

In [125]:
df_cnv = pd.read_csv("../data/MOFA/CNV_factor1.csv")
# df_cnv['gene'] = df_cnv['gene'].map(lambda x:x.replace("_CNV", ""))

In [126]:
df_rna = pd.read_csv("../data/MOFA/RNA_factor1.csv")
# df_rna['gene'] = df_rna['gene'].map(lambda x:x.replace("_RNA", ""))

In [127]:
df_protein = pd.read_csv("../data/MOFA/protein_factor1.csv")
# df_protein['gene'] = df_protein['gene'].map(lambda x:x.replace("_protein", ""))

In [129]:
df_combined = pd.concat([df_cnv, df_rna, df_protein])

df_combined['gene'] = df_combined['symbol'].map(lambda x: x.replace(
    "_rna", "").replace("_cnv", "").replace("_protein", ""))

df_combined['location'] = df_combined['gene'].map(chr_map)

df_combined = df_combined.dropna()

df_combined['Factor1_abs'] = df_combined['Factor1'].abs()


df_combined = df_combined.sort_values(
    by='Factor1_abs',
    ascending=False).reset_index(drop=True).drop(['Factor1_abs'], axis=1)

In [132]:
df_combined['source'] = df_combined['symbol'].map(
    lambda x: 'RNA' if 'rna' in x else 'CNV' if '_cnv' in x else 'protein')

In [140]:
df_combined = df_combined[['gene', 'source', 'location', 'Factor1']]

In [142]:
df_combined.to_csv("../results/MOFA_factor1_combined.csv", index=False)

In [133]:
df_combined[df_combined['symbol'].str.contains("_rna")]

,Factor1,symbol,gene,location,source
865,-0.857216,ATP5J2_rna,ATP5J2,7q22.1,RNA
918,-0.854196,ZNHIT1_rna,ZNHIT1,7q22.1,RNA
926,-0.845275,BCL7B_rna,BCL7B,7q11.23,RNA
1040,-0.806087,PLOD3_rna,PLOD3,7q22.1,RNA
1043,-0.785293,FIS1_rna,FIS1,7q22.1,RNA
...,...,...,...,...,...
43653,0.000013,ADAM17_rna,ADAM17,2p25.1,RNA
43759,0.000009,LRR1_rna,LRR1,14q21.3,RNA
44158,0.000000,MTRNR2L10_rna,MTRNR2L10,Xp11.21,RNA
44160,0.000000,DIRAS2_rna,DIRAS2,9q22.2,RNA


In [145]:
df_combined[df_combined['gene']=='IMPDH1']

,gene,source,location,Factor1
582,IMPDH1,CNV,7q32.1,-1.097114
1246,IMPDH1,RNA,7q32.1,-0.609056
1368,IMPDH1,protein,7q32.1,-0.557443
